In [1]:
import torch
import torch.utils.data
from tqdm.auto import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
from collections import Counter

cuda


In [ ]:
from networks import pretrained_convnextv2_tiny
from dataloader import *

### ConvNextv2-T

In [ ]:
model = pretrained_convnextv2_tiny()
model.eval()
model.to(device)

ConvNeXtV2(
  (downsample_layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm()
    )
    (1): Sequential(
      (0): LayerNorm()
      (1): Conv2d(96, 192, kernel_size=(2, 2), stride=(2, 2))
    )
    (2): Sequential(
      (0): LayerNorm()
      (1): Conv2d(192, 384, kernel_size=(2, 2), stride=(2, 2))
    )
    (3): Sequential(
      (0): LayerNorm()
      (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
    )
  )
  (stages): ModuleList(
    (0): Sequential(
      (0): Block(
        (dwconv): Conv2d(96, 96, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=96)
        (norm): LayerNorm()
        (pwconv1): Linear(in_features=96, out_features=384, bias=True)
        (act): GELU(approximate='none')
        (grn): GRN()
        (pwconv2): Linear(in_features=384, out_features=96, bias=True)
        (drop_path): Identity()
      )
      (1): Block(
        (dwconv): Conv2d(96, 96, kernel_size=(7, 7), s

In [6]:
dataset_val, _= build_imagenet_val_dataset(input_size=224)
imagnet_dataloader_val = torch.utils.data.DataLoader(
    dataset=dataset_val,
    batch_size=1,
    num_workers = 0,
)

Transform = 
Resize(size=256, interpolation=bicubic, max_size=None, antialias=None)
CenterCrop(size=(224, 224))
ToTensor()
Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
---------------------------
reading from datapath data/imagenet
Number of the class = 1000


In [7]:
for inputs, labels in imagnet_dataloader_val:
    print(inputs.shape)
    print(labels.shape)
    break

torch.Size([1, 3, 224, 224])
torch.Size([1])


In [11]:
def validate(model, dataloader):
    correct_top1 = 0
    total_samples = 0
    accumulated_label = 0
    accumulated_predictions = []

    with torch.no_grad():
        for input, label in tqdm(dataloader):

            input, label = input.to('cuda'), label.to('cuda')

            # Forward pass
            output = model(input)

            # Get top-1 predictions
            _, pred_top1 = output.topk(1, dim=1, largest=True, sorted=True)
            
            curr_label = label.item()
            if curr_label==accumulated_label:
                accumulated_predictions.append(pred_top1.item())
            else:
                counter = Counter(accumulated_predictions)
                _, count = counter.most_common(1)[0]
                total_samples += len(accumulated_predictions)
                correct_top1 += count
                accumulated_label = curr_label
                accumulated_predictions = []
    counter = Counter(accumulated_predictions)
    _, count = counter.most_common(1)[0]
    total_samples += len(accumulated_predictions)
    correct_top1 += count
    # Calculate top-1 accuracy percentage
    top1_accuracy = 100 * correct_top1 / total_samples
    return top1_accuracy

top1_accuracy = validate(model, imagnet_dataloader_val)
print(f'Top-1 Accuracy: {top1_accuracy:.2f}%')

  0%|          | 0/50000 [00:00<?, ?it/s]

Top-1 Accuracy: 83.05%


Claimed Accuracy: 83.0%

Tested Accuracy: 83.05%

### ConvNeXt-B

In [2]:
from networks import pretrained_convnextv2_base
from dataloader import *

In [3]:
model = pretrained_convnextv2_base()
model.eval()
model.to(device)

ConvNeXtV2(
  (downsample_layers): ModuleList(
    (0): Sequential(
      (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm()
    )
    (1): Sequential(
      (0): LayerNorm()
      (1): Conv2d(128, 256, kernel_size=(2, 2), stride=(2, 2))
    )
    (2): Sequential(
      (0): LayerNorm()
      (1): Conv2d(256, 512, kernel_size=(2, 2), stride=(2, 2))
    )
    (3): Sequential(
      (0): LayerNorm()
      (1): Conv2d(512, 1024, kernel_size=(2, 2), stride=(2, 2))
    )
  )
  (stages): ModuleList(
    (0): Sequential(
      (0): Block(
        (dwconv): Conv2d(128, 128, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=128)
        (norm): LayerNorm()
        (pwconv1): Linear(in_features=128, out_features=512, bias=True)
        (act): GELU(approximate='none')
        (grn): GRN()
        (pwconv2): Linear(in_features=512, out_features=128, bias=True)
        (drop_path): Identity()
      )
      (1): Block(
        (dwconv): Conv2d(128, 128, kernel_size

In [4]:
dataset_val, _= build_imagenet_val_dataset(input_size=224)
imagnet_dataloader_val = torch.utils.data.DataLoader(
    dataset=dataset_val,
    batch_size=1,
    num_workers = 0,
)

Transform = 
Resize(size=256, interpolation=bicubic, max_size=None, antialias=None)
CenterCrop(size=(224, 224))
ToTensor()
Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
---------------------------
reading from datapath data/imagenet
Number of the class = 1000


In [5]:
def validate(model, dataloader):
    correct_top1 = 0
    total_samples = 0
    accumulated_label = 0
    accumulated_predictions = []

    with torch.no_grad():
        for input, label in tqdm(dataloader):

            input, label = input.to('cuda'), label.to('cuda')

            # Forward pass
            output = model(input)

            # Get top-1 predictions
            _, pred_top1 = output.topk(1, dim=1, largest=True, sorted=True)
            
            curr_label = label.item()
            if curr_label==accumulated_label:
                accumulated_predictions.append(pred_top1.item())
            else:
                counter = Counter(accumulated_predictions)
                _, count = counter.most_common(1)[0]
                total_samples += len(accumulated_predictions)
                correct_top1 += count
                accumulated_label = curr_label
                accumulated_predictions = []
    counter = Counter(accumulated_predictions)
    _, count = counter.most_common(1)[0]
    total_samples += len(accumulated_predictions)
    correct_top1 += count
    # Calculate top-1 accuracy percentage
    top1_accuracy = 100 * correct_top1 / total_samples
    return top1_accuracy

top1_accuracy = validate(model, imagnet_dataloader_val)
print(f'Top-1 Accuracy: {top1_accuracy:.2f}%')

  0%|          | 0/50000 [00:00<?, ?it/s]

Top-1 Accuracy: 85.03%


Claimed Accuracy: 84.9%

Tested Accuracy: 85.03%